In [1]:
#banco vetorial
!pip install pinecone
!pip install -qU langchain-pinecone


#leitura e split
!pip install langchain_community
!pip install pypdf
!pip install langchain_cor
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.4/330.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.


### Lendo dados e criando splits

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
reg_path = 'drive/MyDrive/PodeIsso/regulamento.pdf'


In [14]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(reg_path)
data = loader.load()

In [16]:
# removendo os \n
for doc in data:
    doc.page_content = doc.page_content.replace("\n", "")

In [18]:
#jutando os documentos em um único documento

from langchain_core.documents.base import Document

merged_content = " ".join([doc.page_content for doc in data])
full_document = Document(page_content=merged_content)


In [20]:
#criando splits

import re
from langchain.text_splitter import TextSplitter


class RegulationTextSplitter(TextSplitter):
    def __init__(self, chunk_size=1000, chunk_overlap=0):
        super().__init__(chunk_size, chunk_overlap)
        self.chunk_size = chunk_size

    def split_text(self, text):
        # Regex ajustada para identificar artigos nos formatos "Art. 9º" e "Art. 10."
        article_pattern = re.compile(r"(Art\. \d+[ºo]?.*?)(?=Art\. \d+[ºo]?|$)", re.DOTALL)

        # Encontra todos os artigos e seus textos completos
        articles = article_pattern.findall(text)

        # Agrupar artigos em chunks respeitando o chunk_size
        chunks = []
        current_chunk = []
        current_size = 0

        for article in articles:
            article_size = len(article)
            # Extrair o título do artigo (primeira linha)
            article_title_match = re.match(r"(Art\. \d+[ºo]?)", article)
            article_title = article_title_match.group(1) if article_title_match else ""

            # Caso o artigo inteiro seja maior que o chunk_size
            if article_size > self.chunk_size:
                # Dividir o artigo em pedaços menores
                split_article = [article[i:i + self.chunk_size] for i in range(0, article_size, self.chunk_size)]

                # Marcar os pedaços subsequentes com o título do artigo
                split_article = [
                    (chunk if idx == 0 else f"{article_title} - Continuation: {chunk}")
                    for idx, chunk in enumerate(split_article)
                ]

                # Adicionar pedaços menores ao chunk atual ou diretamente aos chunks
                if current_chunk:  # Salvar o chunk atual se existir
                    chunks.append(" ".join(current_chunk))
                    current_chunk = []
                    current_size = 0
                chunks.extend(split_article)  # Adicionar pedaços do artigo diretamente
                continue

            # Se adicionar o artigo atual excede o chunk_size, finalizar o chunk atual
            if current_size + article_size > self.chunk_size:
                if current_chunk:  # Se houver artigos acumulados, criar o chunk
                    chunks.append(" ".join(current_chunk))
                # Reiniciar para o próximo chunk
                current_chunk = [article]
                current_size = article_size
            else:
                # Adicionar artigo ao chunk atual
                current_chunk.append(article)
                current_size += article_size

        # Adicionar o último chunk, se houver
        if current_chunk:
            chunks.append(" ".join(current_chunk))

        return chunks

# Exemplo de uso
splitter = RegulationTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = splitter.split_text(full_document.page_content)


### Alimentando banco

In [78]:
from langchain_pinecone import PineconeEmbeddings
import os

pinecone_api_key = os.environ["PINECONE_API_KEY"]

model_name = 'multilingual-e5-large'
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=os.environ.get('PINECONE_API_KEY')
)


In [79]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index("regulamento")

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [80]:
from langchain.docstore.document import Document

# Converte os textos em objetos Document
documents = [Document(page_content=text) for text in chunks]

In [81]:
document_ids = vector_store.add_documents(documents=documents)